# Линейная регрессия — Датасет Insurance

Выполнение заданий из файла **01 - Линейная регрессия - ЛР - Insurance (Medical Cost Personal Datasets).pdf**

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model, metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import Lasso

# Настройки отображения
pd.set_option('display.float_format', lambda x: f'{x:.3f}')
sns.set(style="whitegrid", palette="muted")


In [ ]:

# Загружаем данные
data = pd.read_csv("data/insurance.csv")
data.head()


In [ ]:

print("Размерность:", data.shape)
print("\nПропуски:")
print(data.isnull().sum())

data.info()


In [ ]:

# Гистограммы числовых признаков
data.hist(bins=20, figsize=(12, 8))
plt.tight_layout()
plt.show()

# Boxplot для расходов по полу и курению
plt.figure(figsize=(10, 5))
sns.boxplot(x="smoker", y="charges", data=data)
plt.title("Зависимость расходов от курения")
plt.show()


In [ ]:

# Кодируем бинарные признаки
data['smoker'] = data['smoker'].apply(lambda x: 0 if x == 'no' else 1)
data['sex'] = data['sex'].apply(lambda x: 0 if x == 'female' else 1)

# OneHot кодирование региона
data = pd.get_dummies(data, drop_first=True)
data.head()


In [ ]:

features = data.drop('charges', axis=1).columns
X, y = data[features], data['charges']

# Задание 1: train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Размер train:", X_train.shape, "Размер test:", X_test.shape)


In [ ]:

# Задание 2: линейная регрессия
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

print("Свободный член (intercept):", round(lr.intercept_, 2))

# Предсказания
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

# Метрики
def regression_metrics(y_true, y_pred):
    r2 = metrics.r2_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mape = metrics.mean_absolute_percentage_error(y_true, y_pred) * 100
    return r2, mae, mape

print("Train:", regression_metrics(y_train, y_train_pred))
print("Test:", regression_metrics(y_test, y_test_pred))


In [ ]:

# Задание 3: ошибки
train_errors = y_train - y_train_pred
test_errors = y_test - y_test_pred

plt.figure(figsize=(8, 6))
sns.boxplot(data=[train_errors, test_errors])
plt.xticks([0, 1], ["Train", "Test"])
plt.title("Распределение ошибок (y - y_hat)")
plt.show()


In [ ]:

# Задание 4: нормализация + полиномиальные признаки
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

print("Размерность после полиномиальных признаков:", X_train_poly.shape[1])


In [ ]:

# Задание 5: линейная регрессия на полиномиальных признаках
lr_poly = linear_model.LinearRegression()
lr_poly.fit(X_train_poly, y_train)

y_test_poly_pred = lr_poly.predict(X_test_poly)
r2_poly = metrics.r2_score(y_test, y_test_poly_pred)
print("R2 на тесте:", round(r2_poly, 3))


In [ ]:

# Задание 6: коэффициенты модели
coeffs = lr_poly.coef_
print("Количество коэффициентов:", len(coeffs))
print(coeffs[:20])  # выводим первые 20 для примера

print("\nВывод: при больших коэффициентах модель может быть неустойчива, нужна регуляризация.")


In [ ]:

# Задание 7: Lasso-регрессия
lasso = Lasso(alpha=1.0, max_iter=2000)
lasso.fit(X_train_poly, y_train)
y_test_lasso_pred = lasso.predict(X_test_poly)

r2 = metrics.r2_score(y_test, y_test_lasso_pred)
mae = metrics.mean_absolute_error(y_test, y_test_lasso_pred)
mape = metrics.mean_absolute_percentage_error(y_test, y_test_lasso_pred) * 100

print("Метрики Lasso на тесте:")
print("R2:", round(r2, 3))
print("MAE:", round(mae, 0))
print("MAPE:", round(mape, 0), "%")
